#### I do not claim full ownership of following given models, As it is part my work and part inspiriration from various Research Papers and Code Snippets from Github and StackOverflow

In [ ]:
import os
import shutil
import pickle
from time import time
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, PReLU, BatchNormalization, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D, Add, Flatten, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler

In [ ]:
base = '../input/plantvillage-dataset/color/'
training = os.path.join(base)

In [ ]:
N_CLASSES = len(os.listdir(training))
IMG_W = 224
IMG_H = 224

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
                                                training,
                                                target_size=(IMG_W, IMG_H),
                                                class_mode='categorical',
                                                subset='training')

validation_generator = train_datagen.flow_from_directory(
                                                training,
                                                target_size=(IMG_W, IMG_H),
                                                class_mode='categorical',
                                                subset='validation')

In [ ]:
def Alexnet(in_shape=(224,224,3), n_classes= N_CLASSES):
    in_layer = Input(in_shape)
    
    conv1 = Conv2D(96, 11, strides=4, activation='relu')(in_layer)
    pool1 = MaxPool2D(3, 2)(conv1)

    conv2 = Conv2D(256, 5, strides=1, padding='same', activation='relu')(pool1)
    pool2 = MaxPool2D(3, 2)(conv2)
    
    conv3 = Conv2D(384, 3, strides=1, padding='same', activation='relu')(pool2)
    conv4 = Conv2D(256, 3, strides=1, padding='same', activation='relu')(conv3)
    pool3 = MaxPool2D(3, 2)(conv4)
    
    flattened = Flatten()(pool3)
    
    dense1 = Dense(4096, activation='relu')(flattened)
    drop1 = Dropout(0.5)(dense1)
    dense2 = Dense(4096, activation='relu')(drop1)
    drop2 = Dropout(0.5)(dense2)
    
    preds = Dense(n_classes, activation='softmax')(drop2)

    model = Model(in_layer, preds)
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
    return model

In [ ]:
def scheduler(epoch):
    if epoch < 20:
        return 0.001
    elif epoch > 20 and epoch <= 40:
        return 0.0001
    else: 
        return 0.00001
    
tensorboard = TensorBoard(
                        log_dir='logs/{}'.format(int(time())), 
                        histogram_freq=0, 
                        write_graph=True, 
                        write_grads=False, 
                        write_images=True, 
                        embeddings_freq=0, 
                        embeddings_layer_names=None, 
                        embeddings_metadata=None, 
                        embeddings_data=None, 
                        update_freq='epoch')

modelcheckpoint = ModelCheckpoint(
                                filepath = 'model-alexnet-{}.h5'.format(int(time())), 
                                monitor = 'val_acc',
                                verbose = 1,
                                save_best_only = 1,
                                save_weights_only = False,
                                mode = 'max',
                                save_freq = 'epoch' )


lr_scheduler = LearningRateScheduler(scheduler)


callback = [tensorboard, modelcheckpoint, lr_scheduler]
model = Alexnet()
model.summary()
history = model.fit_generator(train_generator, epochs=50, callbacks = callback, verbose = 1, shuffle=True, validation_data = validation_generator)

In [ ]:
def DCNN_Network(in_shape=(224,224,3), n_classes=N_CLASSES):
    in_layer = Input(in_shape)
    
    conv1 = Conv2D(64, 5, strides=1, padding="same", activation='relu')(in_layer)
    conv2 = Conv2D(64, 5, strides=1, padding="same", activation='relu')(conv1)
    bat1 = BatchNormalization()(conv2)
    pool1 = MaxPool2D(pool_size=(2,2),strides=(2,2))(bat1)
    
    conv3 = Conv2D(64, 3, strides=1, padding="same", activation='relu')(pool1)
    conv4 = Conv2D(64, 3, strides=1, padding="same", activation='relu')(conv3)
    bat2 = BatchNormalization()(conv4)
    pool2 = MaxPool2D(pool_size=(2,2),strides=(2,2))(bat2)
    
    conv5 = Conv2D(128, 3, strides=1, padding="same", activation='relu')(pool2)
    conv6 = Conv2D(128, 3, strides=1, padding="same", activation='relu')(conv5)
    bat3 = BatchNormalization()(conv6)
    pool3 = MaxPool2D(pool_size=(2,2),strides=(2,2))(bat3)
    
    conv7 = Conv2D(128, 3, strides=1, padding="same", activation='relu')(pool3)
    conv8 = Conv2D(128, 3, strides=1, padding="same", activation='relu')(conv7)
    bat4 = BatchNormalization()(conv8)
    pool4 = MaxPool2D(pool_size=(2,2),strides=(2,2))(bat4)
    
    conv9 = Conv2D(256, 3, strides=1, padding="same", activation='relu')(pool4)
    conv10 = Conv2D(256, 3, strides=1, padding="same", activation='relu')(conv9)
    bat5 = BatchNormalization()(conv10)
    pool5 = MaxPool2D(pool_size=(2,2),strides=(2,2))(bat5)
    
    
    flattened = Flatten()(pool5)
    dense1 = Dense(512, activation='relu')(flattened)
    dense2 = Dense(512, activation='relu')(dense1)
    preds = Dense(n_classes, activation='softmax')(dense2)

    model = Model(in_layer, preds)
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
    model.summary()
    return model



In [ ]:
def scheduler(epoch):
    if epoch < 20:
        return 0.001
    elif epoch > 20 and epoch <= 40:
        return 0.0001
    else: 
        return 0.00001

tensorboard = TensorBoard(
                        log_dir='logs/{}'.format(int(time())), 
                        histogram_freq=0, 
                        write_graph=True, 
                        write_grads=False, 
                        write_images=True, 
                        embeddings_freq=0, 
                        embeddings_layer_names=None, 
                        embeddings_metadata=None, 
                        embeddings_data=None, 
                        update_freq='epoch')

modelcheckpoint = ModelCheckpoint(
                                filepath = 'model-dcnn-{}.h5'.format(int(time())), 
                                monitor = 'val_acc',
                                verbose = 1,
                                save_best_only = 1,
                                save_weights_only = False,
                                mode = 'max',
                                save_freq = 'epoch' )


lr_scheduler = LearningRateScheduler(scheduler)

callback = [tensorboard, modelcheckpoint, lr_scheduler]
model = DCNN_Network()

history = model.fit_generator(train_generator, epochs=50, callbacks = callback, verbose = 1, shuffle=True, validation_data = validation_generator)

In [ ]:
def makeModel(nbChannels, shape1, shape2, nbClasses, nbRCL=5, nbFilters=128, filtersize = 3):
    model = BuildRCNN(nbChannels, shape1, shape2, nbClasses, nbRCL, nbFilters, filtersize)
    return model

def BuildRCNN(nbChannels, shape1, shape2, nbClasses, nbRCL, nbFilters, filtersize):
    
    def RCL_block(l_settings, l, pool=True, increase_dim=False):
        input_num_filters = l_settings.output_shape[1]
        if increase_dim:
            out_num_filters = input_num_filters*2
        else:
            out_num_filters = input_num_filters

        conv1 = Conv2D(out_num_filters, 1, 1, padding='same')
        stack1 = conv1(l)   	
        stack2 = BatchNormalization()(stack1)
        stack3 = PReLU()(stack2)
        
        conv2 = Conv2D(out_num_filters, filtersize, 1, padding='same', kernel_initializer = 'he_normal')
        stack4 = conv2(stack3)

        stack5 = Add()([stack1, stack4])
        stack6 = BatchNormalization()(stack5)
        stack7 = PReLU()(stack6)
    
        conv3 = Conv2D(out_num_filters, filtersize, 1, padding='same', weights = conv2.get_weights())
        stack8 = conv3(stack7)
        stack9 = Add()([stack1, stack8])
        stack10 = BatchNormalization()(stack9)
        stack11 = PReLU()(stack10)    
        
        conv4 = Conv2D(out_num_filters, filtersize, 1, padding='same', weights = conv2.get_weights())
        stack12 = conv4(stack11)
        stack13 = Add()([stack1, stack12])
        stack14 = BatchNormalization()(stack13)
        stack15 = PReLU()(stack14)    
        
        if pool:
            stack16 = MaxPooling2D((2, 2), (2,2))(stack15) 
            stack17 = Dropout(0.1)(stack16)
        else:
            stack17 = Dropout(0.1)(stack15)
            
        return stack17

    input_img = Input(shape=(shape1, shape2, nbChannels))
    conv_l = Conv2D(nbFilters, 2,2, padding='same', activation='relu')
    l = conv_l(input_img)
    for n in range(nbRCL):
        if n % 2 ==0:
            l = RCL_block(conv_l, l, pool=False)
        else:
            l = RCL_block(conv_l, l, pool=True)
    
    out = Flatten()(l)        
    l_out = Dense(nbClasses, activation = 'softmax')(out)
    
    model = Model(inputs = input_img, outputs = l_out)
    
    return model

model = makeModel(nbChannels = 3, shape1 = IMG_W, shape2 = IMG_H, nbClasses = N_CLASSES, nbRCL=6, nbFilters=96, filtersize = 3)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

In [ ]:
def scheduler(epoch):
    if epoch < 5:
        return 0.001
    elif epoch > 5 and epoch <= 10:
        return 0.0001
    else: 
        return 0.00001

tensorboard = TensorBoard(log_dir="logs1/RCNN-BatchNorm-{}".format(time()), 
                          histogram_freq=0, 
                          write_graph=True,
                          update_freq='epoch')

lr_scheduler = LearningRateScheduler(scheduler)
callbacks = [tensorboard, lr_scheduler]

history = model.fit_generator(train_generator, epochs=20, callbacks = callbacks, verbose = 1, shuffle=True, validation_data = validation_generator)

In [ ]:
class ResNet:
    def __init__(self, n, channels_first=False, initial_lr=0.01, nb_epochs=50):
        self.n = n
        self.initial_lr = initial_lr
        self.nb_epochs = nb_epochs
        self.weight_decay = 0.0005
        self.channels_first = channels_first
        self.data_format = "channels_first" if channels_first else "channels_last"
        self.bn_axis = 1 if channels_first else -1
        self.model = self.make_model()
        self.model.summary()

    def subsumpling(self, output_channels, input_tensor):
        return Conv2D(output_channels, kernel_size=1, strides=(2,2), data_format=self.data_format, kernel_regularizer=l2(self.weight_decay))(input_tensor)

    def block(self, channles, input_tensor):
        shortcut = input_tensor
        x = BatchNormalization(axis=self.bn_axis)(input_tensor)
        x = Activation("relu")(x)
        x = Conv2D(channles, kernel_size=3, padding="same", data_format=self.data_format,kernel_regularizer=l2(self.weight_decay))(x)
        x = BatchNormalization(axis=self.bn_axis)(x)
        x = Activation("relu")(x)
        x = Conv2D(channles, kernel_size=3, padding="same", data_format=self.data_format,kernel_regularizer=l2(self.weight_decay))(x)
        return Add()([x, shortcut])

    def make_model(self):
        input = Input(shape=(3, IMG_W, IMG_H)) if self.channels_first else Input(shape=(IMG_W, IMG_H, 3))
        x = Conv2D(64, kernel_size=3, padding="same", data_format=self.data_format, kernel_regularizer=l2(self.weight_decay))(input)
        for i in range(self.n):
            x = self.block(64, x)
            
        x = self.subsumpling(128, x)
        for i in range(self.n):
            x = self.block(128, x)
        x = self.subsumpling(128, x)
        for i in range(self.n):
            x = self.block(128, x)
            
        x = self.subsumpling(256, x)
        for i in range(self.n):
            x = self.block(256, x)
        x = self.subsumpling(256, x)
        for i in range(self.n):
            x = self.block(256, x)
            
        x = GlobalAveragePooling2D(data_format=self.data_format)(x)
        x = Flatten()(x)
        x = Dense(256, activation="relu")(x)
        x = Dense(N_CLASSES, activation="softmax")(x)

        model = Model(input, x)
        return model
    
    def lr_schduler(self, epoch):
        x = self.initial_lr
        if epoch >= self.nb_epochs * 0.4: x /= 10.0
        if epoch >= self.nb_epochs * 0.6: x /= 10.0
        if epoch >= self.nb_epochs * 0.8: x /= 10.0
        return x

    def train(self, train_generator, validation_generator):
        self.model.compile(optimizer=SGD(lr=self.initial_lr, momentum=0.9), loss="categorical_crossentropy", metrics=["accuracy"])
        train_datagen = ImageDataGenerator(
                            rescale=1./255,
                            validation_split=0.2)

        train_generator = train_datagen.flow_from_directory(
                            PATH,
                            target_size=(IMG_W, IMG_H),
                            class_mode='categorical',
                            subset='training')

        validation_generator = train_datagen.flow_from_directory(
                            PATH,
                            target_size=(IMG_W, IMG_H),
                            class_mode='categorical',
                            subset='validation')

        # Callback
        lr_cb = LearningRateScheduler(self.lr_schduler)
        tensorboard = TensorBoard(log_dir="logs/{}".format(time()),
                          histogram_freq=0,
                          write_graph=True,
                          write_grads=False,
                          write_images=False,
                          embeddings_freq=0,
                          embeddings_layer_names=None,
                          embeddings_metadata=None,
                          embeddings_data=None,
                          update_freq='epoch')

        self.history = self.model.fit_generator(train_generator,
                                           epochs=self.nb_epochs,
                                           steps_per_epoch = 100,
                                           callbacks=[lr_cb, tensorboard],
                                           validation_data = validation_generator)

In [ ]:
net = ResNet(n = 3, nb_epochs=50)
net.train(train_generator, validation_generator)

### MISC CODE

#### Save Training History

In [ ]:
with open('History', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

#### Save Model

In [ ]:
model.save('model.h5')

#### Zip Training Logs

In [ ]:
shutil.make_archive('file', 'zip', 'logs')